# Toy example

Original notebook by Alexander Held

Edited for the MadMiner repository by Johann Brehmer

In this notebook, we use a simple toy example to demonstrate the machine learning and inference algorithms in MadMiner. This allows us to skip many of the more technical steps, there's no dependency on MadGraph, Pythia, or Delphes.

What this tutorial does not do, is explaining the inference methods. To understand what's happening, please have a look at ["Constraining Effective Field Theories With Machine Learning"](https://arxiv.org/abs/1805.00013), which will explain the basic idea in just a few pages. If you really want to get down to the dirty details, ["A Guide to Constraining Effective Field Theories With Machine Learning"](https://arxiv.org/abs/1805.00020) has you covered.

## Preparations

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os
import logging
import numpy as np
from scipy.stats import norm
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline
import corner

from madminer.ml import MLForge

In [2]:
if not os.path.exists('data'):
    os.makedirs('data')

MadMiner uses the Python `logging` module to provide additional information and debugging output. You can choose how much of this output you want to see by switching the level in the following lines to `logging.DEBUG` or `logging.WARNING`.

In [3]:
# MadMiner output
logging.basicConfig(
    format='%(asctime)-5.5s %(name)-20.20s %(levelname)-7.7s %(message)s',
    datefmt='%H:%M',
    level=logging.INFO
)

# Output of all other modules (e.g. matplotlib)
for key in logging.Logger.manager.loggerDict:
    if "madminer" not in key:
        logging.getLogger(key).setLevel(logging.WARNING)

## 1. The simulator

Our simulator depends on just one parameter `theta` and produces one-dimensional observables `x`. It is characterized by one latent variable `z`. "Running" the simulator consists of two steps:
- "Hard process": a value of `z` is drawn from a normal distribution, where the mean depends on `theta`. 
- "Detector": a value for `x` is drawn from a normal distribution with mean equal to `z`. There is no explicit dependence on `theta`.

As in the particle physics case, we assume that we can calculate the joint likelihood ratio `r(x, z | theta0, theta1)` as well as the joint score `t(x, z | theta0, theta1)`, which depend explicitly on `z` (and where the "detector" part cancels).

Here are some general settings (feel free to play with them):

In [4]:
def z_mean(theta):
    return theta + 10.

z_std = 2.
x_std = 1.

The following function runs the simulator for a value of `theta` and calculates the joint likelihood ratio between `theta0` and `theta1` as well as the joint score at `theta_score`:

In [5]:
def simulate(theta, theta0=None, theta1=None, theta_score=None, npoints=None):
    # Draw latent variable
    z = np.random.normal(loc=z_mean(theta), scale=z_std, size=npoints)
    
    # Draw observable
    x = np.random.normal(loc=z, scale=x_std, size=None)
    
    # Calculate joint likelihood ratio and joint score
    if theta0 is not None and theta1 is not None:
        r_xz = norm(loc=z_mean(theta0), scale=z_std).pdf(z) / norm(loc=z_mean(theta1), scale=z_std).pdf(z)
    else:
        r_xz = None
        
    if theta_score is not None:
        t_xz = (x - z_mean(theta_score)) / z_std**2
    else:
        t_xz = None
    
    return x, r_xz, t_xz

Usually the likelihood function `p(x|theta)` is intractable, but in this toy example it is just given by the convolution of two Gaussians, which is again a Gaussian. We will use this to validate the results later:

In [6]:
def calculate_likelihood_ratio(x, theta0, theta1=0.):
    combined_std = (z_std**2 + x_std**2)**0.5
    r_x = norm(loc=z_mean(theta0), scale=combined_std).pdf(x) / norm(loc=z_mean(theta1), scale=combined_std).pdf(x)
    return r_x

## 2. Run the simulation and generate training data

First, we define the parameter points we want to use for training:

In [7]:
n_param_points = 100000 # number of parameter points to train

theta0 = np.random.uniform(low=-4.0, high=4.0, size=n_param_points) # numerator, uniform prior
theta1 = np.zeros(shape=n_param_points)                             # denominator: fixed at 0

We then run the simulator (one sample per parameter point):

In [11]:
# Sample from theta0
x_from_theta0, r_xz_from_theta0, t_xz_from_theta0 = simulate(theta0, theta0, theta1, theta0)

# Sample from theta1
x_from_theta1, r_xz_from_theta1, t_xz_from_theta0 = simulate(theta1, theta0, theta1, theta0)

# Combine results and reshape
x_train = np.hstack((x_from_theta0, x_from_theta1)).reshape(-1,1)
r_xz_train = np.hstack((x_from_theta0, x_from_theta1)).reshape(-1,1)
t_xz_train = np.hstack((x_from_theta0, x_from_theta1)).reshape(-1,1)
y_train = np.hstack((np.zeros_like(x_from_theta0), np.ones_like(np.ones_like(x_from_theta1)))).reshape(-1,1)
theta0_train = np.hstack((theta0, theta0)).reshape(-1,1)


Save everything to files.

In [12]:
np.save('data/theta0_train.npy', theta0_train)
np.save('data/x_train.npy', x_train)
np.save('data/y_train.npy', y_train)
np.save('data/r_xz_train.npy', r_xz_train)
np.save('data/t_xz_train.npy', t_xz_train)

## 3. Model training

Let's train a single neural network to estimate the likelihood ratio, using the ALICES method introduced in ["Likelihood-free inference with an improved cross-entropy estimator"](https://arxiv.org/abs/1808.00973).

In [13]:
forge = MLForge()

forge.train(
    method='alices',
    x_filename='data/x_train.npy',
    y_filename='data/y_train.npy',
    theta0_filename='data/theta0_train.npy',
    r_xz_filename='data/r_xz_train.npy',
    t_xz0_filename='data/t_xz_train.npy',
    alpha=0.1,
    n_epochs=20,
    n_hidden=(20,20),
    validation_split=0.3,
    batch_size=128,
)

forge.save('models/alices')

15:38 madminer.ml          INFO    Starting training
15:38 madminer.ml          INFO      Method:                 alices
15:38 madminer.ml          INFO      Training data: x at data/x_train.npy
15:38 madminer.ml          INFO                     theta0 at data/theta0_train.npy
15:38 madminer.ml          INFO                     y at data/y_train.npy
15:38 madminer.ml          INFO                     r_xz at data/r_xz_train.npy
15:38 madminer.ml          INFO                     t_xz (theta0) at  data/t_xz_train.npy
15:38 madminer.ml          INFO      Features:               all
15:38 madminer.ml          INFO      Method:                 alices
15:38 madminer.ml          INFO      Hidden layers:          (20, 20)
15:38 madminer.ml          INFO      Activation function:    tanh
15:38 madminer.ml          INFO      Batch size:             128
15:38 madminer.ml          INFO      Trainer:                amsgrad
15:38 madminer.ml          INFO      Epochs:                 20
15:38 madm

KeyboardInterrupt: 

## 4. Generate evaluation data

We generate one test set from theta = 1:

In [ ]:
n_param_points_test = 10000 # number of parameter points to test

theta_test = 1. * np.ones(shape=n_param_points_test).reshape(-1,1)

Run the simulator to get observations:

In [ ]:
x_test, _, _ = simulate(theta_test)

np.save('data/x_test.npy', x_test)

We want to evaluate the expected likelihood ratio on a range of parameter points:

In [ ]:
theta_grid = np.linspace(-5.,5.,100).reshape(-1, 1)

np.save('data/theta_grid.npy', theta_grid)

In this toy example, we can calculate the true likelihood ratio. We will save the expected log likelihood ratio (multiplied with a conventional factor of -2):

In [ ]:
nllr_test_true = []

for theta in theta_grid:
    r = calculate_likelihood_ratio(x_test, theta)
    nllr_test_true.append(-2. * np.mean(np.log(r)))

## 5. Evaluation

It's time to evaluate our likelihood ratio estimator!

In [ ]:
forge = MLForge(debug=False)
forge.load('models/model_toy')

log_r, _, _ = forge.evaluate(
    theta0_filename='data/theta_grid.npy',
    x='data/x_test.npy',
    evaluate_score=False
)

nllr_test_alices = -2. * np.mean(log_r, axis=1)

This is the expected log likelihood ratio over parameter space:

In [ ]:
best_fit = float(theta_grid[np.argmin(nllr_test_alices)])

fig = plt.figure(figsize=(5,5))

plt.plot(theta_grid, nllr_test_true, label='Ground truth')
plt.plot(theta_grid, nllr_test_alices, label='ALICES')

plt.xlabel(r'$\theta$')
plt.ylabel(r'$\mathbb{E}_x [ -2\, \log \,r(x | \theta, \theta_{1}) ]$')

plt.legend()
plt.tight_layout()
plt.show()

print("Best fit point: theta = {}".format(best_fit))
print("True:           theta = {}".format(1.))

That's it!